In [1]:
from urllib import request
import json
from io import StringIO
import os

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re


from nltk.stem import PorterStemmer 
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import model_from_json

import webbrowser

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import sys
import random
from IPython.display import JSON

pd.options.display.max_colwidth = 100
%matplotlib inline

#demoji.download_codes()
#access_token = os.environ['ACCESS_TOKEN']


import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prayt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# stemmed_words_dset = {'dataset': dataset}
# with open('/content/drive/My Drive/stemmed_words_dset.pickle', 'wb') as f:
#     pickle.dump(stemmed_words_dset, f, pickle.HIGHEST_PROTOCOL)

# Predictions added for WHOLE dataset, binary 1/0 also added for data that was prelabeled r/d also added to dset
# predictions_dataset = {'dataset': dataset}
# with open('/content/drive/My Drive/predictions_dataset.pickle', 'wb') as f:
#     pickle.dump(predictions_dataset, f, pickle.HIGHEST_PROTOCOL)

# predictions_plus_stemmed_rnn_ready_texts = {'dataset': dataset}
# with open('/content/drive/My Drive/predictions_plus_stemmed_rnn_ready_texts.pickle', 'wb') as f:
#     pickle.dump(predictions_plus_stemmed_rnn_ready_texts, f, pickle.HIGHEST_PROTOCOL)

# with open('/content/drive/My Drive/keyword_dset.pickle', 'rb') as f:
#     a_pickle = pickle.load(f)
# other_dataset = a_pickle['keyword_dset']
# # # congress = a_pickle['congress']

# with open('/content/drive/My Drive/stemmed_words_dset.pickle', 'rb') as f:
#     a_pickle = pickle.load(f)
# dataset = a_pickle['dataset']

# with open('/content/drive/My Drive/predictions_dataset.pickle', 'rb') as f:
#     a_pickle = pickle.load(f)
# other_dataset = a_pickle['dataset']

with open('predictions_plus_stemmed_rnn_ready_texts.pickle', 'rb') as f:
    a_pickle = pickle.load(f);
dataset = a_pickle['dataset']

with open('rnn_model_3.pickle', 'rb') as f:
    lstm_pickle = pickle.load(f);

W0917 15:22:10.877324 21408 deprecation_wrapper.py:119] From C:\Users\prayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 15:22:34.018769 21408 deprecation_wrapper.py:119] From C:\Users\prayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 15:22:34.917285 21408 deprecation_wrapper.py:119] From C:\Users\prayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0917 15:22:49.072157 21408 deprecation_wrapper.py:119] From C:\Users\prayt\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0917 15:22:49.144138 21408 deprecation.py:506

In [3]:
def clean_and_stem(text):
    text = str(text)
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()
    ps = PorterStemmer()
    stemmed_words = [ps.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text


In [4]:
def predict_text_lean(a_string):
    # USER_INPUT_STRING - must be string, text preprocessing not necessary
    print('user input:')
    print(a_string, '\n')
    MAX_SEQUENCE_LENGTH = 50

    lstm_model = lstm_pickle['model']
    lstm_tokenizer = lstm_pickle['tokenizer']

    a_test = lstm_tokenizer.texts_to_sequences([a_string])
    a_padded_test = pad_sequences(a_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post')


    predictions = lstm_model.predict(x=a_padded_test)
    predictions = predictions.flatten()

    for i in list(range(len(predictions))):  
        print('input content partisan certainty', predictions[i])
        if predictions[i] > .5:  
          predictions[i] = 1
          print('prediction: left lean')
        else:
          predictions[i] = 0
          print('prediction: right lean')

    user_predict = int(predictions)
    return user_predict

In [50]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_fit = tfidf.fit_transform(dataset['ad_stemmed_body'])

def alt_tfidf_recommendations(a_string, df, predicted_lean):
    #tfidf = TfidfVectorizer(stop_words='english')
    #tfidf_fit = tfidf.fit_transform(df['ad_stemmed_body'])
    a_string = clean_and_stem(a_string)
    string_vector = tfidf.transform([a_string]) 

    cosine_similarities = linear_kernel(string_vector, tfidf_fit).flatten()
    probabilities_array = df['probability'].to_numpy()
    try:
        if predicted_lean == 0:
            composite_score = cosine_similarities*(probabilities_array**25)
        elif predicted_lean == 1:
            composite_score = cosine_similarities*((1-probabilities_array)**25)
        else:
            print('predicted_lean is not registering as a zero or one integer')
    except:
        print('something is wrong in composite score calculation if statement')
    composite_score[0] = -1
    recommend_indices = composite_score.argsort()[-2:]
    print('cosine similarity between source ad and served ads:', cosine_similarities[recommend_indices])
    print('composite score partisan certainty and cosine similarity', composite_score[recommend_indices])
    print('probabilities array val', probabilities_array[recommend_indices])
    return recommend_indices


def alt_lean_and_generate_predictions(input_index, df, match_on='ad_creative_body'):   
    ad_lean = predict_text_lean(df.ad_creative_body[input_index])  #predict_text_lean should return a zero(r) or 1(d)
    recommend_indices = alt_tfidf_recommendations(df.ad_creative_body[input_index], df, ad_lean)
    recommends = df[match_on][recommend_indices] 
    
    # [::-1] reverses order of recommend_indices so tabs open in opposite order
    for ad_index in recommend_indices[::-1]:
        #ad_2_url = df.ad_snapshot_url[ad_index]
        webbrowser.open(df.ad_snapshot_url[ad_index])
        if df.probability[ad_index] > .5:
            print('served content', ad_index, 'predicted lean: left wing')
        else:
            print('served content', ad_index, 'predicted lean: right wing')
        print(df.probability[ad_index])
    ad_1_url = df.ad_snapshot_url[input_index]
    webbrowser.open(ad_1_url)
    print('\n','served content text')
    return recommends

# Recommend Ads
the function below takes in a number representing an ad in the database and outputs an ad that is topically relevant but from the other end of the political spectrum.

In [51]:
any_number_between_1_and_157000 = 35235

alt_lean_and_generate_predictions(any_number_between_1_and_157000, dataset)

user input:
Last week I introduced a new bipartisan bill to repeal the Death Tax. This bill gives farmers and ranchers, as well as many others, permanent relief from estate taxes and gets rid of one of the most egregious federal taxes we still have on the books – a tax on death. 

The Tax Cuts and Jobs Act was able to temporary lift the ceiling on the estate tax, but that ceiling begins to fall in the near future and because of arcane US Senate rules, wasn’t made permanent. The solution is to repeal the death tax once and for all. It’s an outdated tax which hurts many Missouri farmers and families. 

My Estate Tax Repeal Bill gives certainty to hard-working farmers. With the passage of this bill, farmers will be able to divert funds away from estate planning and towards things like business investment and equipment and livestock purchases. 

Now, when we lose a loved one, we won’t be subject to a financial fiasco. We won’t have to worry if our farm will stay within our family or if the

105063    Sanders plans to introduce an estate tax plan with rates up to 77 percent. The "For the 99.8% Ac...
68307     Many Americans owe additional taxes or have smaller refunds this Tax Day, after Trump and the GO...
Name: ad_creative_body, dtype: object